#### Changing directory to root directory

In [1]:
import os

In [2]:
%pwd

'/home/akshay/Desktop/chichen_diseases/research'

In [3]:
%cd ..

/home/akshay/Desktop/chichen_diseases


### Initializing DataIngestionConfig class

In [4]:
from dataclasses import dataclass
from pathlib import Path

# this is the configuration for config/config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from src.chicken_disease.constants import *
from src.chicken_disease.utils.common import create_directories, read_yaml

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        # These point to constants dir
        # where config.yaml and params.yaml are located

        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # artifacts_root come from config/config.yaml which returns "artifacts
        # hence a directory will be created names "artifacts"
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_configuration(self) -> DataIngestionConfig:
        # we are saving config for data ingestion found in config.yaml in "config" variable
        config = self.config.data_ingestion

        # config.root_dir = artifacts/data_ingestion as per config.yaml
        create_directories([config.root_dir])

        # this is creating the config object from
        # DataIngestionConfig class above
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file, 
            unzip_dir = config.unzip_dir 
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from src.chicken_disease.logging import logger
from src.chicken_disease.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    # we are storing DataIngestionConfig object in "config"
    # we need to download the file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists, Size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file to the specified folder
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



## Data Ingestion Pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_configuration()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


# This will download the data, unzip the data and will display size of the data

[2023-06-06 15:29:12,203 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2023-06-06 15:29:12,207 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-06-06 15:29:12,209 : INFO : common : created directory at: artifacts]
[2023-06-06 15:29:12,213 : INFO : common : created directory at: artifacts/data_ingestion]
[2023-06-06 15:29:20,998 : INFO : 3114768457 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F08:5AF5:6C5B55:7DC573:647F0371
Accept-Ranges: bytes
Date: Tue, 06 Jun 2023 09:59:15 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4746-B